# Loading libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  LSTM, Dense, Normalization, Dropout, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow import keras
import kerastuner
from kerastuner.tuners import Hyperband
from tensorflow.keras import backend as K
import tensorflow as tf
import csv

In [ ]:
merged_data_short = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/merged_data_short.csv")


# Sampling 50 stations for hyperparametertuning

In [ ]:
# Sampling 50 unique station IDs at random
random_stations = merged_data_short['station_id_encoded'].drop_duplicates().sample(n=50, random_state=42)

# Include these stations from the dataset
merged_data_short_sample = merged_data_short[merged_data_short['station_id_encoded'].isin(random_stations)]


# Splitting data

In [ ]:
# Sort by 'date'
merged_data_short_sorted = merged_data_short_sample.sort_values(by=['date', 'station_id_encoded', 'hour'])

merged_data_short_sorted.drop(columns=["station_name", "short_name"], inplace=True)

merged_data_short_sorted.reset_index(drop=True, inplace=True)

# Split sorted data into features (X) and target (y)
X_sorted = merged_data_short_sorted.drop('start_count', axis=1)
y_sorted = merged_data_short_sorted['start_count']

# Splitting on date
train_end_index = X_sorted[X_sorted['date'] == '2022-11-30'].index[-1]
val_end_index = X_sorted[X_sorted['date'] == '2023-03-10'].index[-1]

# Split the data manually
X_train = X_sorted.iloc[:train_end_index]
y_train = y_sorted.iloc[:train_end_index]
X_val = X_sorted.iloc[train_end_index:val_end_index]
y_val = y_sorted.iloc[train_end_index:val_end_index]
X_test = X_sorted.iloc[val_end_index:]
y_test = y_sorted.iloc[val_end_index:]


In [ ]:
X_train.drop(columns=["date"], inplace=True)
X_val.drop(columns=["date"], inplace=True)
X_test.drop(columns=["date"], inplace=True)


# Standardizing columns

In [ ]:
# Feature columns to be standardized
feature_columns = ["hour", "capacity", "longitude", "latitude", "temperature_2m (°C)",
                   "relativehumidity_2m (%)", "precipitation (mm)", "snowfall (cm)",
                   "cloudcover (%)", "direct_radiation (W/m²)", "windspeed_10m (km/h)",
                   "bike_lane_length_km", "restaurants_count", "rail_stations_count",
                   "universities_count", "bus_stations_count", "businesses_count", "parks_count"]

# Create the Normalization layer and adapt it to the training data
standardizer = tf.keras.layers.Normalization(axis=-1)
standardizer.adapt(X_train[feature_columns])

# Apply the standardizer to the training data
X_train[feature_columns] = standardizer(X_train[feature_columns].values)

# Apply the standardizer to the validation data
X_val[feature_columns] = standardizer(X_val[feature_columns].values)

# Apply the standardizer to the test data
X_test[feature_columns] = standardizer(X_test[feature_columns].values)

# Create the Normalization layer and adapt it to the training data
target_standardizer = tf.keras.layers.Normalization(axis=-1)
target_standardizer.adapt(y_train[['start_count']])

# Apply the standardizer to the training data target variable
y_train['start_count'] = target_standardizer(y_train[['start_count']])

# Apply the standardizer to the validation data target variable
y_val['start_count'] = target_standardizer(y_val[['start_count']])

# Apply the standardizer to the test data target variable
y_test['start_count'] = target_standardizer(y_test[['start_count']])



# Batch generator hyperparametertuning

In [ ]:
def batch_generator(X, y, time_steps=24, batch_size=256, infinite_loop=True):
    total_size = len(X) - time_steps
    start_idx = 0

    while True:
        X_batch = np.zeros((batch_size, time_steps, X.shape[1]))
        y_batch = np.zeros((batch_size,))

        for i in range(batch_size):
            if start_idx + time_steps <= total_size:
                X_batch[i] = X.iloc[start_idx:start_idx + time_steps].values
                y_batch[i] = y.iloc[start_idx + time_steps]
                start_idx += 1
            else:
                if infinite_loop:
                    start_idx = 0
                else:
                    break

        yield (X_batch, y_batch)

        if not infinite_loop and start_idx + time_steps > total_size:
            break

# Small batch size to keep memory usage low
batch_size = 256

train_gen = batch_generator(X_train, y_train, time_steps=24, batch_size=256, infinite_loop=True)
val_gen = batch_generator(X_val, y_val, time_steps=24, batch_size=256, infinite_loop=True)
test_gen = batch_generator(X_test, y_test, time_steps=24, batch_size=256, infinite_loop=False)

# Test the generator to see if it yields batches correctly
X_batch, y_batch = next(train_gen)
(X_batch.shape, y_batch.shape)


((256, 24, 42), (256,))

# Hyperparametertuning LSTM model

## Defining LSTM model

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

input_shape = (24, X_batch.shape[2])

def build_lstm_model(hp):
    num_units = hp.Int('num_units', min_value=32, max_value=128, step=32)
    drop_out = hp.Choice('drop_out', [0.2, 0.4, 0.6])
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    lstm_layers = hp.Choice('lstm_layers', [1, 2])
    include_dense = hp.Choice('include_dense', [False, True])

    model = Sequential()
    model.add(LSTM(units=num_units, return_sequences=True if lstm_layers == 2 else False, input_shape=input_shape))
    model.add(Dropout(drop_out))

    if lstm_layers == 2:
        model.add(LSTM(units=num_units))
        model.add(Dropout(drop_out))

    if include_dense:
        dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=32)
        model.add(Dense(dense_units, activation='relu'))

    model.add(Dense(1))

    # Define the optimizer
    if optimizer_choice == 'adam':
        learning_rate = hp.Choice('adam_learning_rate', [0.1, 0.01, 0.001])
        optimizer = Adam(learning_rate=learning_rate, clipvalue=1.0)
    elif optimizer_choice == 'sgd':
        learning_rate = hp.Choice('sgd_learning_rate', [0.1, 0.01, 0.001])
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer_choice == 'rmsprop':
        learning_rate = hp.Choice('rmsprop_learning_rate', [0.1, 0.01, 0.001])
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', rmse])

    model.summary()
    return model

## Tuning using Hyperband and batch generator

In [ ]:
tuner = Hyperband(
    build_lstm_model,
    objective='val_loss',
    max_epochs=5,
    directory='/content/drive/MyDrive/Colab Notebooks/model_7',
    project_name='hyperband_lstm_tuning',
    factor=3,
    hyperband_iterations=2

)
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
# Calculate the number of steps per epoch
steps_per_epoch = (len(X_train) - 24) // batch_size
validation_steps = (len(X_val) - 24) // batch_size

# During the hyperparameter tuning search, pass these as arguments
tuner.search(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=5,
    callbacks=[stop_early]
)

In [ ]:
# Get the hyperparameters of the best model
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print('Best hyperparameters:', best_hyperparameters.values)

## Saving hyperparameters

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]

# Define the file name for the CSV
filename = '/content/drive/MyDrive/Colab Notebooks/best_hyperparameters.csv'

# Open the file in write mode
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Hyperparameter', 'Value'])
    for key, value in best_hyperparameters.values.items():
        csvwriter.writerow([key, value])

# Hyperparametertuning CNN-LSTM model

## Defining CNN-lSTM model

In [ ]:
# Define RMSE function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Define Input shape
input_shape = (24, X_batch.shape[2])


def build_cnn_lstm_model(hp):
    model = Sequential()
    model.add(Conv1D(
        filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
        activation='relu',
        input_shape=input_shape
    ))

    # Optional Second Conv1D layer
    C2 = hp.Int('filters_2', min_value=0, max_value=128, step=32)
    if C2 > 0:
        model.add(Conv1D(
            filters=C2,
            kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
            activation='relu'
        ))

    model.add(LSTM(
        units=hp.Int('units', min_value=30, max_value=90, step=20),
        activation='relu'
    ))

    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.6, step=0.2)))

    # Optional Additional Dense layer(s)
    for i in range(hp.Int('num_dense_layers', 1, 2)):
        model.add(Dense(
            units=hp.Int(f'dense_{i+1}_units', min_value=10, max_value=100, step=20),
            activation='relu'
        ))

    model.add(Dense(1))

    # Optimizer choice and learning rate setup remains the same
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    if optimizer_choice == 'adam':
        learning_rate = hp.Choice('adam_learning_rate', [0.1, 0.01, 0.001])
        optimizer = Adam(learning_rate=learning_rate, clipvalue=1.0)
    elif optimizer_choice == 'sgd':
        learning_rate = hp.Choice('sgd_learning_rate', [0.1, 0.01, 0.001])
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer_choice == 'rmsprop':
        learning_rate = hp.Choice('rmsprop_learning_rate', [0.1, 0.01, 0.001])
        optimizer = RMSprop(learning_rate=learning_rate)

    # Compile model
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', rmse])

    model.summary()
    return model


## Tuning using Hyperband and batch generator

In [ ]:
# Initialize the Hyperband tuner
tuner = Hyperband(
    build_cnn_lstm_model,
    objective='val_loss',
    max_epochs=5,
    directory="/content/drive/MyDrive/Colab Notebooks/Good/CNN_LSTM_Tuning_2",
    project_name='hyperband_lstm_tuning',
    factor=3,
    hyperband_iterations=2
)

In [ ]:
# Calculate the number of steps per epoch
steps_per_epoch = (len(X_train) - 24) // batch_size
validation_steps = (len(X_val) - 24) // batch_size

tuner.search(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=5,
    callbacks=[stop_early]
)

In [ ]:
# Get the hyperparameters of the best model
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print('Best hyperparameters:', best_hyperparameters.values)

## Saving hyperparameters

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]

# Define the file name for the CSV
filename = '/content/drive/MyDrive/Colab Notebooks/best_hyperparameters.csv'

# Open the file in write mode
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Hyperparameter', 'Value'])
    for key, value in best_hyperparameters.values.items():
        csvwriter.writerow([key, value])
